In [1]:
import tensorflow as tf
import pandas as pd
import functools
import time

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, roc_auc_score, \
    auc, average_precision_score, pairwise_distances
import scikitplot as skplt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import pickle
import dill
from functools import partial
from keras.utils import np_utils

C:\Users\user\Miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\user\Miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\user\Miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\user\Miniconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passi

# read gene features

In [2]:
genes = pd.read_csv("data/gene_data/gene_features_all.csv",index_col=0)

# read the training,val,test splits

In [3]:
all_df = pd.read_csv("data/gene_data/allsigs.csv",index_col=0)
val1_df = pd.read_csv("data/gene_data/splits/csv/val_set_1.csv",index_col=0)
val2_df = pd.read_csv("data/gene_data/splits/csv/val_set_2.csv",index_col=0)
val3_df = pd.read_csv("data/gene_data/splits/csv/val_set_3.csv",index_col=0)
val4_df = pd.read_csv("data/gene_data/splits/csv/val_set_4.csv",index_col=0)
test_df = pd.read_csv("data/gene_data/splits/csv/test_set.csv",index_col=0)

valsets = [val1_df,val2_df,val3_df,val4_df]

# only keep labels with more than 3 examples?

In [4]:
all_df = all_df[all_df["moa_count"]>3]
genes = genes.loc[all_df.index]

# Encode labels

In [5]:
# encode class values as integers
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(all_df['moa_v1'])
#encoded_Y = encoder.transform(all_df['moa_v1'])
# convert integers to dummy variables (i.e. one hot encoded)
#dummy_y = np_utils.to_categorical(encoded_Y)

LabelEncoder()

# first remove the test set from all_df and all genes

In [6]:
test_genes = genes.loc[test_df["sig_id"]]
test_sigs = all_df.loc[test_df["sig_id"]]
all_df = all_df.drop(test_df["sig_id"])
genes = genes.drop(test_df["sig_id"])

# Parameter space

In [7]:
# defining the space
fspace = {
    'dense_size_1' : hp.quniform('dense_size_1', 512,1024,64), #+
    'dense_size_2' : hp.quniform('dense_size_2', 256,512,64), #+
    'dense_size_3' : hp.quniform('dense_size_3', 128,256,64), #+
    'lr' : hp.uniform('lr', 0.0005, 0.01),
    'dropout' : hp.uniform('dropout',0.1,0.5),
    'epochs' : hp.quniform('epochs',5,50,5),
}

In [8]:
import tensorflow as tf
import keras
import keras.backend as K
from keras.layers import Dense, Dropout, Input, Lambda
from keras.models import Model, Sequential, load_model
from keras.callbacks import History, ReduceLROnPlateau,EarlyStopping,ModelCheckpoint

In [13]:
def objective(fspace, all_df, genes, valsets, label_encoder):
    accs = []
    model_params = {
        'dense_size' : [int(fspace['dense_size_1']),int(fspace['dense_size_2']),int(fspace['dense_size_3'])],
        'dropout' : [fspace['dropout'],fspace['dropout'],fspace['dropout']],
        'lr' : fspace['lr']
    }
    for i in range(len(valsets)):
        #split val and train
        val_genes = genes.loc[valsets[i]["sig_id"]]
        val_sigs = all_df.loc[valsets[i]["sig_id"]]
        train_genes = genes.drop(valsets[i]["sig_id"])
        train_sigs = all_df.drop(valsets[i]["sig_id"])
        #one hot encode labels
        encoded_Y_train = label_encoder.transform(train_sigs['moa_v1'])
        encoded_Y_val = label_encoder.transform(val_sigs['moa_v1'])
        train_y = np_utils.to_categorical(encoded_Y_train)
        val_y = np_utils.to_categorical(encoded_Y_val)
        # define the mlp model
        
        gene_input = Input(name = 'gene_input',shape = (978,), dtype = 'float32')

        fc1 = Dense(model_params['dense_size'][0], activation = 'relu', kernel_initializer='random_normal')(gene_input)
        fc1 = Dropout(model_params['dropout'][0])(fc1)
        fc2 = Dense(model_params['dense_size'][1], activation = 'relu', kernel_initializer='random_normal')(fc1)
        fc2 = Dropout(model_params['dropout'][1])(fc2)
        fc3 = Dense(model_params['dense_size'][2], activation = 'relu', kernel_initializer='random_normal')(fc2)
        fc3 = Dropout(model_params['dropout'][2])(fc3)

        prediction = Dense(all_df['moa_v1'].nunique(), activation = 'softmax')(fc3)
        adam = keras.optimizers.Adam(lr=model_params["lr"], beta_1=0.9, beta_2=0.999, decay=0.0, amsgrad=False)
        mlp_model = Model(inputs = gene_input, outputs = prediction)
        mlp_model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
        
        rlr = ReduceLROnPlateau(monitor='loss',factor=0.5, patience=4, verbose=1, min_lr=0.0000001)
        
        mlp_model.fit(np.array(train_genes),train_y, batch_size = 64, 
                      epochs = int(fspace['epochs']), 
                      verbose = 0, shuffle = True, validation_data = None)
       
        pred = mlp_model.predict(np.array(val_genes))
        
        accs.append(accuracy_score(np.argmax(val_y,axis=1), np.argmax(pred,axis=1)))
    ave_acc = np.mean(accs,axis = 0)
    return {'loss': -ave_acc ,  'status': STATUS_OK}

In [14]:
fmin_objective = partial(objective, all_df = all_df, genes = genes, valsets = valsets, label_encoder = label_encoder)

In [15]:
def run_trials():

    trials_step = 100  # how many additional trials to do after loading saved trials. 1 = save after iteration
    max_trials = 1  # initial max_trials. put something small to not have to wait

    
    try:  # try to load an already saved trials object, and increase the max
        trials = pickle.load(open("MLP_genes.hyperopt", "rb"))
        print("Found saved Trials! Loading...")
        max_trials = len(trials.trials) + trials_step
        print("Rerunning from {} trials to {} (+{}) trials".format(len(trials.trials), max_trials, trials_step))
    except:  # create a new trials object and start searching
        trials = Trials()

    best = fmin(fn = fmin_objective, space = fspace, algo=tpe.suggest, max_evals=max_trials, trials=trials)

    print("Best:", best)
    
    # save the trials object
    with open("MLP_genes.hyperopt", "wb") as f:
        pickle.dump(trials, f)
    return(trials)

In [16]:
trials = run_trials()

Found saved Trials! Loading...
Rerunning from 1 trials to 101 (+100) trials
100%|██████████████████████████████████████████| 101/101 [4:42:00<00:00, 167.53s/trial, best loss: -0.6265860190229938]
Best: {'dense_size_1': 576.0, 'dense_size_2': 256.0, 'dense_size_3': 128.0, 'dropout': 0.2733924105790248, 'epochs': 25.0, 'lr': 0.000981699073272445}


# load the best parameters

In [19]:
model_params = {
        'dense_size' : [int(896),int(384),int(128)],
        'dropout' : [0.45,0.45,0.45],
        'lr' : 0.00127
    }

In [20]:
accs = []
test_preds = []
for i in range(len(valsets)):
    #split val and train
    val_genes = genes.loc[valsets[i]["sig_id"]]
    val_sigs = all_df.loc[valsets[i]["sig_id"]]
    train_genes = genes.drop(valsets[i]["sig_id"])
    train_sigs = all_df.drop(valsets[i]["sig_id"])
    #one hot encode labels
    encoded_Y_train = label_encoder.transform(train_sigs['moa_v1'])
    encoded_Y_val = label_encoder.transform(val_sigs['moa_v1'])
    train_y = np_utils.to_categorical(encoded_Y_train)
    val_y = np_utils.to_categorical(encoded_Y_val)
    # define the mlp model
        
    gene_input = Input(name = 'gene_input',shape = (978,), dtype = 'float32')

    fc1 = Dense(model_params['dense_size'][0], activation = 'relu', kernel_initializer='random_normal')(gene_input)
    fc1 = Dropout(model_params['dropout'][0])(fc1)
    fc2 = Dense(model_params['dense_size'][1], activation = 'relu', kernel_initializer='random_normal')(fc1)
    fc2 = Dropout(model_params['dropout'][1])(fc2)
    fc3 = Dense(model_params['dense_size'][2], activation = 'relu', kernel_initializer='random_normal')(fc2)
    fc3 = Dropout(model_params['dropout'][2])(fc3)

    prediction = Dense(all_df['moa_v1'].nunique(), activation = 'softmax')(fc3)
    adam = keras.optimizers.Adam(lr=model_params["lr"], beta_1=0.9, beta_2=0.999, decay=0.0, amsgrad=False)
    mlp_model = Model(inputs = gene_input, outputs = prediction)
    mlp_model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
        
    rlr = ReduceLROnPlateau(monitor='loss',factor=0.5, patience=4, verbose=1, min_lr=0.0000001)
        
    mlp_model.fit(np.array(train_genes),train_y, batch_size = 64, 
                    epochs = int(100), 
                    verbose = 1, shuffle = True, validation_data = None)
       
    pred = mlp_model.predict(np.array(val_genes))
    test_pred = mlp_model.predict(np.array(test_genes))
    test_preds.append(test_pred)
    accs.append(accuracy_score(np.argmax(val_y,axis=1), np.argmax(pred,axis=1)))
ave_acc = np.mean(accs,axis = 0)

Epoch 1/100
2087/2087 [==============================] - 1s 554us/step - loss: 5.4621 - acc: 0.0738
Epoch 2/100
2087/2087 [==============================] - 0s 163us/step - loss: 4.3158 - acc: 0.1433
Epoch 3/100
2087/2087 [==============================] - 0s 187us/step - loss: 4.0477 - acc: 0.1773
Epoch 4/100
2087/2087 [==============================] - 0s 169us/step - loss: 3.8741 - acc: 0.2080
Epoch 5/100
2087/2087 [==============================] - 0s 174us/step - loss: 3.6897 - acc: 0.2362
Epoch 6/100
2087/2087 [==============================] - 0s 193us/step - loss: 3.5622 - acc: 0.2487
Epoch 7/100
2087/2087 [==============================] - 0s 192us/step - loss: 3.3846 - acc: 0.2750
Epoch 8/100
2087/2087 [==============================] - 0s 170us/step - loss: 3.2615 - acc: 0.2899
Epoch 9/100
2087/2087 [==============================] - 0s 164us/step - loss: 3.1259 - acc: 0.3138
Epoch 10/100
2087/2087 [==============================] - 0s 185us/step - loss: 2.9534 - acc: 0.3340

2087/2087 [==============================] - 0s 146us/step - loss: 0.7359 - acc: 0.8232
Epoch 82/100
2087/2087 [==============================] - 0s 152us/step - loss: 0.7281 - acc: 0.8088
Epoch 83/100
2087/2087 [==============================] - 0s 169us/step - loss: 0.6517 - acc: 0.8347
Epoch 84/100
2087/2087 [==============================] - 0s 143us/step - loss: 0.5804 - acc: 0.8380
Epoch 85/100
2087/2087 [==============================] - 0s 158us/step - loss: 0.6912 - acc: 0.8227
Epoch 86/100
2087/2087 [==============================] - 0s 172us/step - loss: 0.6313 - acc: 0.8380
Epoch 87/100
2087/2087 [==============================] - 0s 159us/step - loss: 0.6370 - acc: 0.8352
Epoch 88/100
2087/2087 [==============================] - 0s 167us/step - loss: 0.6162 - acc: 0.8347
Epoch 89/100
2087/2087 [==============================] - 0s 172us/step - loss: 0.6740 - acc: 0.8376
Epoch 90/100
2087/2087 [==============================] - 0s 176us/step - loss: 0.6237 - acc: 0.8366
Epo

2141/2141 [==============================] - 0s 167us/step - loss: 0.9629 - acc: 0.7567
Epoch 63/100
2141/2141 [==============================] - 0s 172us/step - loss: 0.9735 - acc: 0.7450
Epoch 64/100
2141/2141 [==============================] - 0s 191us/step - loss: 0.9251 - acc: 0.7609
Epoch 65/100
2141/2141 [==============================] - 0s 166us/step - loss: 0.9501 - acc: 0.7571
Epoch 66/100
2141/2141 [==============================] - 0s 172us/step - loss: 0.9479 - acc: 0.7511
Epoch 67/100
2141/2141 [==============================] - 0s 184us/step - loss: 0.8961 - acc: 0.7613
Epoch 68/100
2141/2141 [==============================] - 0s 154us/step - loss: 0.8897 - acc: 0.7702
Epoch 69/100
2141/2141 [==============================] - 0s 171us/step - loss: 0.8215 - acc: 0.7842
Epoch 70/100
2141/2141 [==============================] - 0s 176us/step - loss: 0.8765 - acc: 0.7707
Epoch 71/100
2141/2141 [==============================] - 0s 133us/step - loss: 0.8413 - acc: 0.7711
Epo

2108/2108 [==============================] - 0s 139us/step - loss: 1.2375 - acc: 0.6769
Epoch 44/100
2108/2108 [==============================] - 0s 173us/step - loss: 1.1836 - acc: 0.6988
Epoch 45/100
2108/2108 [==============================] - 0s 163us/step - loss: 1.1850 - acc: 0.6788
Epoch 46/100
2108/2108 [==============================] - 0s 183us/step - loss: 1.1598 - acc: 0.7002
Epoch 47/100
2108/2108 [==============================] - 0s 111us/step - loss: 1.1232 - acc: 0.7021
Epoch 48/100
2108/2108 [==============================] - 0s 147us/step - loss: 1.0758 - acc: 0.7116
Epoch 49/100
2108/2108 [==============================] - 0s 153us/step - loss: 1.0528 - acc: 0.7201
Epoch 50/100
2108/2108 [==============================] - 0s 178us/step - loss: 1.0179 - acc: 0.7225
Epoch 51/100
2108/2108 [==============================] - 0s 144us/step - loss: 1.0383 - acc: 0.7310
Epoch 52/100
2108/2108 [==============================] - 0s 175us/step - loss: 1.0622 - acc: 0.7206
Epo

2141/2141 [==============================] - 0s 161us/step - loss: 1.8734 - acc: 0.5427 0s - loss: 1.8353 - acc: 0
Epoch 25/100
2141/2141 [==============================] - 0s 172us/step - loss: 1.8812 - acc: 0.5353
Epoch 26/100
2141/2141 [==============================] - 0s 150us/step - loss: 1.7515 - acc: 0.5647
Epoch 27/100
2141/2141 [==============================] - 0s 178us/step - loss: 1.7291 - acc: 0.5787
Epoch 28/100
2141/2141 [==============================] - 0s 179us/step - loss: 1.6886 - acc: 0.5736
Epoch 29/100
2141/2141 [==============================] - 0s 166us/step - loss: 1.6471 - acc: 0.5838
Epoch 30/100
2141/2141 [==============================] - 0s 172us/step - loss: 1.5517 - acc: 0.6044
Epoch 31/100
2141/2141 [==============================] - 0s 172us/step - loss: 1.5906 - acc: 0.5993
Epoch 32/100
2141/2141 [==============================] - 0s 177us/step - loss: 1.4841 - acc: 0.6207
Epoch 33/100
2141/2141 [==============================] - 0s 164us/step - los

In [21]:
accs

[0.6470588235294118,
 0.5846153846153846,
 0.6122448979591837,
 0.6307692307692307]

# Test performance

In [25]:
encoded_Y_test = label_encoder.transform(test_sigs['moa_v1'])
test_y = np_utils.to_categorical(encoded_Y_test)
test_predictions = np.argmax(np.mean(test_preds,axis = 0),axis=1)

In [26]:
accuracy_score(np.argmax(test_y,axis=1), test_predictions)

0.5042735042735043

In [27]:
test_predictions

array([ 31,  31,  31,  31,  50,  50,  50,  44,  44,  50,  44,  44,  44,
        44,  50,  44,  44,  44,  44,  44,  50,  50,  50,  50,  50,  50,
        50,  50,  50,  50,  50,  50,  50,  50,  50,  50,  50,  50,  52,
        52, 102,  52,  69, 102,  89,   4, 102,  52,  63,  63,  63,  63,
        63,  10,  10,  10,  35,  78,  93,  93,  80,  80,  80,  80,  80,
        93,  80,  80,  80,  80,  80,  80,  17,  14,  14,  58,  14,  14,
       114,  14,  14,  77,  14,  14,  14,  14,  14,  14,  14,  78,  14,
        14,  14,  58,  50,  14,  44,  50,  44,  44,  50,  44,  44,  44,
        44,  44,  62,  45,  18,  46,  45, 102, 102, 102, 119, 104, 104],
      dtype=int64)

In [30]:
test_sigs['predicted'] = label_encoder.inverse_transform(test_predictions.astype(int))
drug_acc = per_drug_acc(test_sigs)

In [31]:
drug_acc

0.625

In [28]:
def per_drug_acc(df):
    unique_drugs = df['rdkit'].unique()
    s = 0
    for drug in unique_drugs:
        filt = df[df['rdkit']==drug]
        score = accuracy_score(filt['moa_v1'], filt['predicted'])
        nunique_moa = filt['predicted'].nunique()
        if score >= (1/nunique_moa):
            s = s + 1
    return(s/len(unique_drugs))


In [90]:
#le.inverse_transform(pred.astype(int))